In [ ]:
!pip install torch

##1. Check PyTorch version

In [3]:
#check version
import torch
print(torch.__version__)

2.9.0+cu126


## 2. Tensor Storage Basics

#### A tensor in PyTorch has TWO layers:
*  (1) STORAGE: a flat 1-D block of memory (all tensor values stored linearly)
*  (2) VIEW / TENSOR OBJECT: uses shape, stride, and offset to interpret the storage


In [ ]:
# Creating a 3x3 tensor

points = torch.tensor(
    [[9, 7, 5],
     [4, 8, 9],
     [2, 4, 5]],
    dtype=torch.float32)


### points.storage()
* #### Shows the underlying 1-D memory block that stores the numbers.
* #### Even though the tensor is 3x3, storage is always FLAT.

In [18]:
print("Tensor Storage (flat memory):")
print(points.storage())


Tensor Storage (flat memory):
 9.0
 7.0
 5.0
 4.0
 8.0
 9.0
 2.0
 4.0
 5.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 9]


#### TensorFlow DOES NOT have .storage(), because its memory model is different.

In [20]:
print("Tensor Shape:", points.size())

Tensor Shape: torch.Size([3, 3])


### points.storage_offset()
*  Shows the index inside the storage where the tensor STARTS reading.
* For a normal tensor (not sliced), this is always 0.

In [21]:
print("Storage Offset:", points.storage_offset())


Storage Offset: 0


### 3. Slicing and Storage Sharing Example

In [14]:
# Create a 1-D tensor of values 0..9
x = torch.arange(10)

####Slice elements index 3 to 7 (5 elements)
###### IMPORTANT:
* y does NOT get new memory!
* It shares the SAME STORAGE as x.

In [22]:
y = x[3:8]

In [25]:
print("Storage of x:", x.storage())

Storage of x:  0
 1
 2
 3
 4
 5
 6
 7
 8
 9
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 10]


In [26]:
print("Storage of y (shared with x):", y.storage())

Storage of y (shared with x):  0
 1
 2
 3
 4
 5
 6
 7
 8
 9
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 10]


#### y starts at index 3 of the shared storage.

In [27]:
print("y Storage Offset:", y.storage_offset())

y Storage Offset: 3


## 4.STRIDE

### Stride tells PyTorch: how many steps to jump in storage to reach the next element in each dimension.

#### Think of stride as the distance in memory between:

* Next column

* Next row

* Next depth (if 3D)

In [28]:
t = torch.tensor([[9,7,5],
                  [4,8,9],
                  [2,4,5]])
t.stride()

(3, 1)

In [30]:
#Storage of the tensor:
t.storage()

 9
 7
 5
 4
 8
 9
 2
 4
 5
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]

#### Stride logic:

* To go to next column, move +1 step → correct

* To go to next row, move +3 steps → correct

So PyTorch can walk through storage correctly.

### Transpose swaps rows ↔ columns.

In [36]:
print(t)
print("*************")

tT = t.t()
print(tT)
tT.stride()


tensor([[9, 7, 5],
        [4, 8, 9],
        [2, 4, 5]])
*************
tensor([[9, 4, 2],
        [7, 8, 4],
        [5, 9, 5]])


(1, 3)

#### Transpose doesn’t change storage; it only changes how PyTorch walks through the same memory.
Row steps become +1 (0→1→2) and column steps become +3 (0→3→6), so stride = (1, 3).

Storage of original tensor t:

[9,7,5,4,8,9,2,4,5]
 index: 0 1 2 3 4 5 6 7 8

* Row 0 of t.t() = 9, 4, 2 → storage indexes: 0 → 3 → 6 (step = +3)

* Col 0 of t.t() = 9, 7, 5 → storage indexes: 0 → 1 → 2 (step = +1)

Hence stride = (1, 3).

In [38]:
points

tensor([[9., 7., 5.],
        [4., 8., 9.],
        [2., 4., 5.]])

In [46]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [45]:
print(torch.cuda.is_available())

False


In [47]:
points.to(device)

tensor([[9., 7., 5.],
        [4., 8., 9.],
        [2., 4., 5.]])